## Онтологическое моделирование. Социальная сеть

Теоретический материал по построению онтологии представлен в ноутбуке в папке Task_9. Into/Example ontology.ipynb


Создадим онтологию для социальной сети. 

Представим, что мы постепенно собираем данные с некоторой социальной сети и добавляем их в онтологию. После каждого изменения, мы обязательно проверяем, что все работает.

Вам не нужно собирать данные, все данные есть в папке Task_9. Onto/data:

- users.csv: пользователи соцсети.
- posts.csv: посты (контент) с заголовками.
- categories.csv: тематические классы (Fact, Opinion, FakeNews, Meme и т.д.).
- publishes.csv: кто опубликовал какой пост.


Пример онтологии: ИИ в социальных сетях

🧱 Классы:
- User — пользователь соцсети.
- Post — публикация.
- Category — базовая категория контента.
- FakeNews, Opinion, Fact, Meme, Educational — подклассы категорий.

🔗 Свойства:
- has_published(User → Post)
- has_title(Post)
- has_category(Post → Category)

🧍 Примеры пользователей:
- alice публикует clickbait_ai_news и openai_announces_gpt5
- bob публикует ai_is_dangerous, funny_ai_cat, how_neural_networks_work

📌 Категории контента:
- clickbait_ai_news — FakeNews
- ai_is_dangerous — Opinion
- openai_announces_gpt5 — Fact
- funny_ai_cat — Meme



In [1]:
# Установка необходимых библиотек
!pip install --no-cache-dir owlready2 rdflib SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 32.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 561.7 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.48-cp312-cp312-linux_x86_64.whl size=23882552 sha256=762d382257fa996b42384463ffe070e01ab24cbc0946dffc457ea37cd706847f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ngcdgvio/wheels/80/6b/6e/3a4fd869625821d573b27fc501b24fd715ab53ee483e981664
Successfully built owlready2

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from owlready2 import *
import pandas as pd


# Загружаем данные из CSV файлов
users_df = pd.read_csv("Task_9.Onto/data/users.csv")
posts_df = pd.read_csv("Task_9.Onto/data/posts.csv")
categories_df = pd.read_csv("Task_9.Onto/data/categories.csv")
publishes_df = pd.read_csv("Task_9.Onto/data/publishes.csv")


In [3]:
users_df

id       name
0    u1    Алексей
1    u2      Мария
2    u3       Иван
3    u4  Екатерина
4    u5    Николай
5    u6      Ольга
6    u7    Дмитрий
7    u8  Анастасия
8    u9     Сергей
9   u10      Елена
10  u11      Роман

In [4]:
posts_df 

id                                              title
0    p1          Новый ИИ от OpenAI способен писать музыку
1    p2     Илон Маск планирует имплантировать чипы в мозг
2    p3             Почему стоит опасаться глубоких фейков
3    p4                 ИИ ошибся и выдал фейковую новость
4    p5     ИИ улучшает качество фото с низким разрешением
5    p6          Советы по защите личных данных в соцсетях
6    p7           Мнение: будущее за гибридным интеллектом
7    p8  Факт: ИИ уже используется в медицинской диагно...
8    p9                     Ироничный мем про ИИ и роботов
9   p10                 Новости: ИИ победил чемпиона по го
10  p11                    Мнение: роботы не заменят людей

# Итерация 1

## Задание 1: Создайте онтологию для двух классов

Для создания концептов используйте файлы: 
- posts.csv (содержит: id, title) -> класс Post
- users.csv (содержит: id, name) -> класс User

Для создания отношений используйте файл publishers.csv (содержит: user_id, post_id — кто опубликовал пост):
- объектное свойство "опубликовал" (has_published) между User и Post
- для Post свойство-значение "имеет заголовок" (has_title) строкового типа
- для User свойство-значение "имеет имя" (has_name) строкового типа

In [5]:
onto = get_ontology("http://example.org/social_ai.owl")

In [6]:
with onto:
    # Определяем классы
    class User(Thing):
        pass
    
    class Post(Thing):
        pass
    
    # Определяем свойства
    class has_published(ObjectProperty):
        domain = [User]
        range = [Post]
    
    class has_name(DataProperty):
        domain = [User]
        range = [str]
    
    class has_title(DataProperty):
        domain = [Post]
        range = [str]

## Задание 2: Создайте экземпляры пользователей
Создайте всех пользователей на основе User из набора данных users_df.

Передайте доступные свойства для экземпляров. 

In [7]:
# Загружаем данные
users_df = pd.read_csv("Task_9.Onto/data/users.csv")

with onto:
    # Создаем экземпляры пользователей
    for index, row in users_df.iterrows():
        user = onto.User(row['id'])
        user.has_name = [row['name']]

## Задание 3: Создайте экземпляры постов
Создайте все посты на основе Post из набора данных posts_df.

Передайте доступные свойства для экземпляров. 

In [8]:
# Загружаем данные
posts_df = pd.read_csv("Task_9.Onto/data/posts.csv")

with onto:
    # Создаем экземпляры постов
    for index, row in posts_df.iterrows():
        post = onto.Post(row['id'])
        post.has_title = [row['title']]

## Задание 4: Создайте связи с помощью отношения  has_published между пользователями и постами

In [9]:
# Загружаем данные
publishes_df = pd.read_csv("Task_9.Onto/data/publishes.csv")

with onto:
    # Создаем связи has_published
    for index, row in publishes_df.iterrows():
        user = onto[row['user_id']]
        post = onto[row['post_id']]
        user.has_published.append(post)

## Задание 5: Проверка онтологии

Проверьте все ли работает. Выведите для каждого пользователя название опубликованного  им поста. 

In [10]:
# Выводим информацию
for user in onto.User.instances():
    print(f"Пользователь: {user.has_name[0]}")
    for post in user.has_published:
        print(f"  Опубликовал пост: {post.has_title[0]}")

Пользователь: Алексей
  Опубликовал пост: Новый ИИ от OpenAI способен писать музыку
Пользователь: Мария
  Опубликовал пост: Илон Маск планирует имплантировать чипы в мозг
Пользователь: Иван
  Опубликовал пост: Почему стоит опасаться глубоких фейков
Пользователь: Екатерина
  Опубликовал пост: ИИ ошибся и выдал фейковую новость
Пользователь: Николай
  Опубликовал пост: ИИ улучшает качество фото с низким разрешением
Пользователь: Ольга
  Опубликовал пост: Советы по защите личных данных в соцсетях
Пользователь: Дмитрий
  Опубликовал пост: Мнение: будущее за гибридным интеллектом
Пользователь: Анастасия
  Опубликовал пост: Факт: ИИ уже используется в медицинской диагностике
Пользователь: Сергей
  Опубликовал пост: Ироничный мем про ИИ и роботов
Пользователь: Елена
  Опубликовал пост: Новости: ИИ победил чемпиона по го
Пользователь: Роман
  Опубликовал пост: Мнение: роботы не заменят людей


# Итерация 2

## Задание 6: Добавьте новый концепт в существующую онтологию

Добавьте класс Category в уже существующую онтологию

In [11]:
with onto:
    # Определяем класс Category
    class Category(Thing):
        pass

## Задание 7: Добавьте свойства и связи

После добавления класса Category добавьте объектное свойство has_category для связи постов с категориями.

In [12]:
with onto:
    # Определяем свойство has_category
    class has_category(ObjectProperty):
        domain = [Post]
        range = [Category]

## Задание 8: Создайте экземпляры категорий 

Создайте экземпляяры категорий и реализуйте связь между постами и категориями

In [13]:
# Загружаем данные
categories_df = pd.read_csv("Task_9.Onto/data/categories.csv")

with onto:
    # Создаем экземпляры категорий
    unique_categories = categories_df['category'].unique()
    for category_name in unique_categories:
        onto.Category(category_name)
    
    # Связываем посты с категориями
    for index, row in categories_df.iterrows():
        post = onto[row['post_id']]
        category = onto[row['category']]
        post.has_category.append(category)

## Задание 9: Проверка онтологии

Проверьте все ли работает. Выведите для каждого пользователя название опубликованного  им поста с указанием к какой катеригории этот пост относится.
Примечание. Для каждого поста должна быть выведена только одна категория, без повторов

In [14]:
# Выводим информацию
for user in onto.User.instances():
    print(f"Пользователь: {user.has_name[0]}")
    for post in user.has_published:
        category = post.has_category[0] if post.has_category else "Без категории"
        print(f"  Пост: {post.has_title[0]}, Категория: {category.name}")

Пользователь: Алексей
  Пост: Новый ИИ от OpenAI способен писать музыку, Категория: Fact
Пользователь: Мария
  Пост: Илон Маск планирует имплантировать чипы в мозг, Категория: Fact
Пользователь: Иван
  Пост: Почему стоит опасаться глубоких фейков, Категория: Opinion
Пользователь: Екатерина
  Пост: ИИ ошибся и выдал фейковую новость, Категория: FakeNews
Пользователь: Николай
  Пост: ИИ улучшает качество фото с низким разрешением, Категория: Fact
Пользователь: Ольга
  Пост: Советы по защите личных данных в соцсетях, Категория: Opinion
Пользователь: Дмитрий
  Пост: Мнение: будущее за гибридным интеллектом, Категория: Opinion
Пользователь: Анастасия
  Пост: Факт: ИИ уже используется в медицинской диагностике, Категория: Fact
Пользователь: Сергей
  Пост: Ироничный мем про ИИ и роботов, Категория: Meme
Пользователь: Елена
  Пост: Новости: ИИ победил чемпиона по го, Категория: Fact
Пользователь: Роман
  Пост: Мнение: роботы не заменят людей, Категория: Opinion


## Задание 10: Создание аксиомы

Для данной онтологии, которая представляет посты пользователей в социальных сетях, можно добавить аксиому, которая будет утверждать, что <b>"Каждый пост, опубликованный пользователем, имеет хотя бы одну категорию"</b>. Это гарантирует, что все посты будут связаны с какой-то категорией.

Чтобы выразить это в OWL-формате, можно создать аксиому, которая будет связывать посты с категорией через свойство has_category. Также необходимо избегать повторов. 

In [15]:
with onto:
    # Добавляем аксиому: каждый пост должен иметь хотя бы одну категорию
    class Post(Thing):
        equivalent_to = [Thing & has_category.some(Category)]

## Задание 11: Проверка онтологии

Проверьте, что аксиома работает -> для каждого поста проверяется, что он имеет категорию и выводится название самой категории. 

In [16]:
# Проверяем посты и их категории
for post in onto.Post.instances():
    category = post.has_category[0] if post.has_category else "Без категории"
    print(f"Пост: {post.has_title[0]}, Категория: {category.name}")

Пост: Новый ИИ от OpenAI способен писать музыку, Категория: Fact
Пост: Илон Маск планирует имплантировать чипы в мозг, Категория: Fact
Пост: Почему стоит опасаться глубоких фейков, Категория: Opinion
Пост: ИИ ошибся и выдал фейковую новость, Категория: FakeNews
Пост: ИИ улучшает качество фото с низким разрешением, Категория: Fact
Пост: Советы по защите личных данных в соцсетях, Категория: Opinion
Пост: Мнение: будущее за гибридным интеллектом, Категория: Opinion
Пост: Факт: ИИ уже используется в медицинской диагностике, Категория: Fact
Пост: Ироничный мем про ИИ и роботов, Категория: Meme
Пост: Новости: ИИ победил чемпиона по го, Категория: Fact
Пост: Мнение: роботы не заменят людей, Категория: Opinion


# Итерация 3

## Задание 12: Создание подкласса для FakeNews

Создайте потомка FakeNewsPost для класса Post

In [17]:
with onto:
    # Определяем подкласс FakeNewsPost
    class FakeNewsPost(Post):
        pass

## Задание 13: Применение подкласса к постам с категорией 'FakeNews'

Если категория поста - 'FakeNews', то сделайте его экземпляром FakeNewsPost

In [18]:
# Загружаем данные
categories_df = pd.read_csv("Task_9.Onto/data/categories.csv")

with onto:
    # Присваиваем тип FakeNewsPost постам с категорией FakeNews
    for index, row in categories_df.iterrows():
        if row['category'] == 'FakeNews':
            post = onto[row['post_id']]
            post.is_a.append(onto.FakeNewsPost)

## Задание 14: Проверка онтологии

Выполните проверку -  есть ли посты типа FakeNewsPost, если есть, то выведите их названия (title)


In [19]:
# Проверяем посты типа FakeNewsPost
for post in onto.FakeNewsPost.instances():
    print(f"FakeNewsPost: {post.has_title[0]}")

FakeNewsPost: ИИ ошибся и выдал фейковую новость


# Итерация 4

## Задание 15: Инверсивное отношение

Создайте инверсивное объектное отношение  "имеет авторство" has_author для Post и User  отношению has_published.

Примечание. Для указания, что отношение инверсивное другому отношению используйте свойство inverse_property. 

Например, 

```
with onto:
   class has_child(ObjectProperty):
        domain = [Parent]
        range = [Children]
        inverse_property = has_parent
```

In [20]:
with onto:
    # Определяем инверсивное свойство has_author
    class has_author(ObjectProperty):
        domain = [Post]
        range = [User]
        inverse_property = has_published

 ## Задание 15: Cоздайте концепт LegalCase - Судебное разбирательство

 Мы хотим выявить всех, кто "строчит" фейки.
 В онтологии это можно выразить с помощью правила, связывающего факт публикации фейковой новости с последствием — судебным разбирательством. 

 Создайте класс LegalCase.

In [21]:
with onto:
    # Определяем класс LegalCase
    class LegalCase(Thing):
        pass

 ## Задание 16: Добавим объектное свойство involved_in_case

 Добавим объектное свойство involved_in_case между классами User и LegalCase.


In [22]:
with onto:
    # Определяем свойство involved_in_case
    class involved_in_case(ObjectProperty):
        domain = [User]
        range = [LegalCase]

Примечание. Добавим правило: 

`если User публикует NewsPost с категорией FakeNews, то он участвует в LegalCase`

Чтобы указать такое правило можно использовать синтаксис SWRL правил через Imp:

```
with onto:
    rule = Imp("FakeNewsLegalImp", namespace=onto)
    rule.set_as_rule("""
        User(?u), Post(?p), hasAuthor(?p, ?u), has_category(?p, FakeNews) 
        -> involved_in_case(?u, ?case), LegalCase(?case)
    """)

```

Данное правило мы можем загрузить, однако нам необходим ризонер, который позволяет выявлять на основе заданных правил новые знания. К примеру, если пользователь опубликовал пост с категорией FakeNews, то он автоматически (при инференции) будет связан с новым инстансом LegalCase.

Правило SWRL, которое задано выше, не будет автоматически "выполняться" или выводить результаты в owlready2. Это связано с тем, что owlready2 не имеет встроенного механизма выполнения (reasoning) SWRL-правил — только их хранение и сериализация в OWL.

Чтобы правило дало видимый результат, нужно запустить ризонер, но в Simba его установить не получится (проблема с Java). 

Поэтому нужно "сэмулировать" выполнение SWRL-правил вручную в Python. 

## Задание 17: Реализуйте в ручную применение вышеописанной SWRL-логики

Реализуйте проверку для всех пользователей:

если User публикует NewsPost с категорией FakeNews, то он участвует в LegalCase (судебном разбирательстве).

In [23]:
# Загружаем данные
categories_df = pd.read_csv("Task_9.Onto/data/categories.csv")
publishes_df = pd.read_csv("Task_9.Onto/data/publishes.csv")

with onto:
    # Применяем логику SWRL вручную
    for index, row in categories_df.iterrows():
        if row['category'] == 'FakeNews':
            post_id = row['post_id']
            # Находим пользователя, опубликовавшего этот пост
            user_row = publishes_df[publishes_df['post_id'] == post_id]
            if not user_row.empty:
                user_id = user_row.iloc[0]['user_id']
                user = onto[user_id]
                # Создаем экземпляр LegalCase
                legal_case = onto.LegalCase(f"case_{post_id}")
                # Связываем пользователя с делом
                user.involved_in_case.append(legal_case)

## Задание 18: Сохраните полученную онтологию 

 Сохраните полученную онтологию, используя метод save(). Пример, 

```
 onto.save(file="onlogy.owl", format="rdfxml")

```

Откройте файл с онтологией, изучите вид представления данных.


In [24]:
# Сохраняем онтологию
onto.save(file="ontology.owl", format="rdfxml")

## Задание 19: Отнологический граф

Постройте онтологический граф для полученной онтологии. 
Важно, чтобы полученный граф был читаемый!

In [26]:
from graphviz import Digraph

# Создаем граф
dot = Digraph(comment='Ontology Graph')
dot.attr(rankdir='BT')  # Направление снизу вверх для лучшей читаемости

# Добавляем классы
for cls in onto.classes():
    dot.node(cls.name, cls.name, shape='box')

# Добавляем связи между классами (подклассы)
for cls in onto.classes():
    for parent in cls.is_a:
        if isinstance(parent, Thing.__class__):
            dot.edge(cls.name, parent.name, label='is_a')

# Добавляем объектные свойства
for prop in onto.object_properties():
    domain = prop.domain[0].name if prop.domain else 'Thing'
    range_ = prop.range[0].name if prop.range else 'Thing'
    dot.edge(domain, range_, label=prop.name)

# Сохраняем и отображаем граф
dot.render('ontology_graph', format='png', cleanup=True)
print("Граф сохранен в файл ontology_graph.png")

Граф сохранен в файл ontology_graph.png


##  Задание 20: Итерация 5

Добавьте новую итерацию для полученной онтологии. В ней можно добавить новые классы, отношения, правила и т.д. 

Проверьте онтологию с новыми изменениями. 
